In [2]:
from selenium import webdriver
from time import sleep
import os
from bs4 import BeautifulSoup
import requests
import shutil
from xlsxwriter import Workbook

class App:
    def __init__(self,username='user_name',password='password',target_username='user_name',targetpath='C:/Users/seanp/Desktop/Web Scrapping/instaphotos'):
        self.username=username
        self.password=password
        self.path=targetpath
        self.error=False
        self.target_username=target_username
        chrome_options = webdriver.ChromeOptions()
        prefs = {"profile.default_content_setting_values.notifications" : 2}
        chrome_options.add_experimental_option("prefs",prefs)
        self.driver = webdriver.Chrome('C:/Users/seanp/Downloads/chromedriver_win32/chromedriver.exe',chrome_options=chrome_options)
        self.main_url ='https://www.instagram.com'
        self.driver.get(self.main_url)
        sleep(3)
        
        self.log_in()
        if self.error is False:
            self.close_box()
            self.open_target_profile()
        if self.error is False:
            sleep(3)
            self.scroll_down()
        if self.error is False:
            if not os.path.exists(targetpath):
                os.mkdir(path)
            self.download_images()
            
            
            
            
        self.driver.close()
    def download_captions(self,images):
        caption_folder_path = os.path.join(self.path, 'captions')
        if not os.path.exists(caption_folder_path):
            os.mkdir(caption_folder_path)
        self.write_captions_to_excel(images,caption_folder_path)
        for index, image in enumerate(images):
            try:
                caption = image['alt']
            except KeyError:
                caption='No captions for this image'
            filename = 'caption_' + str(index) + '.txt'
            file_path=os.path.join(caption_folder_path, filename)
            print('Downloading Caption Now', index)
            link=image['src']
            with open(file_path,'wb') as file:
                file.write(str('link:' +str(link) + '\n' + 'caption:' +caption).encode())
        
    def download_images(self):
        soup=BeautifulSoup(self.driver.page_source,'lxml')
        all_images= soup.find_all('img')
        self.download_captions(all_images)
        print(len(all_images))
        for index, image in enumerate(all_images):
            filename ='image_' +str(index) +'.jpg'
            #image_path = self.path + '/' + filename
            image_path =os.path.join(self.path, filename)
            link=image['src']
            print('Downloading Image Now', index)
            try:
                response=requests.get(link, stream=True)
                with open(image_path,'wb') as file:
                    shutil.copyfileobj(response.raw,file)
            except Exception as e:
                print('Coukd download file',index)
                print('Image Link',link)
                print(e)
    def write_captions_to_excel(Self, images, caption_path):
        print('Writing to excel')
        workbook= Workbook(caption_path+'/captions.xlsx')
        workbook = Workbook(os.path.join(caption_path, 'caption.xlsx'))
        worksheet= workbook.add_worksheet()
        row=0
        worksheet.write(row,0,'Image name')
        worksheet.write(row,1,'Caption')
        row+=1
        for index, image in enumerate(images):
            filename = 'image_'+str(index)+'.jpg'
            try:
                caption = image['alt']
            except KeyError:
                cpation = 'No caption exists'
            worksheet.write(row,0,filename)
            worksheet.write(row,1,caption)
            row+=1
        workbook.close()
        
        
    def open_target_profile(self):
        try:
            sleep(7)
            print()
            search_bar =self.driver.find_element_by_xpath('//input[@placeholder="Search"]')
            sleep(4)
            search_bar.send_keys(self.target_username)
            sleep(3)
            target_profile =self.main_url + '/' +self.target_username + '/'
            self.driver.get(target_profile)
            sleep(5)
        except Exception:
            self.error=True
            print('Couldnt find search bar')

    def scroll_down(self):
        try:
            no_of_posts=self.driver.find_element_by_xpath('//span[@class="g47SY "]')
            no_of_posts =int(str(no_of_posts.text).replace(',','')) #15,483->15483
            if no_of_posts>6:
                no_of_scrolls = int(no_of_posts/6) - 4
                for value in range(no_of_scrolls):
                    self.driver.execute_script('window.scrollTo(0, document.body.scrollHeight);')
                    sleep(3)
            sleep(12)
        except Exception as e:
            print('Couldnt find no of posts while scrolling')
            self.error=True
            print(e)
    
    def close_box(self):
        try:
            sleep(7)
            close_button=self.driver.find_element_by_xpath('//button[@class="sqdOP  L3NKy   y3zKF     "]')
            sleep(4)
            close_button.click()
#             sleep(7)
#             close_button_1=self.driver.find_element_by_xpath('//button[@class="aOOlW  bIiDR  "]')
#             sleep(4)
#             close_button_1.click()
#             sleep(5)
        except Exception:
            pass
    
    def resolve_multiple_tabs(self):
        try:
            self.driver.switch_to.window(self.driver.window_handles[1])
            self.driver.close()
            self.driver.switch_to.window(self.driver.window_handles[0])
            
        except Exception:
            pass
            
        
    def log_in(self):
        try:
            try:
                user_name=  self.driver.find_element_by_xpath('//input[@class="_2hvTZ pexuQ zyHYP"]')
                user_name.send_keys(self.username)
                password=  self.driver.find_element_by_xpath('//input[@aria-label="Password"]')
                password.send_keys(self.password)
                password.submit()
                #self.resolve_multiple_tabs()
            except Exception:
                print('Some exception occurred while putting username and password fields')
                self.error=True
        except Exception:
            self.error=True
            print('Unable to find log in button')
        
if __name__ == '__main__':
    app=App()

<ipython-input-2-be54b8277af9>:19: DeprecationWarning: use options instead of chrome_options
  self.driver = webdriver.Chrome('C:/Users/seanp/Downloads/chromedriver_win32/chromedriver.exe',chrome_options=chrome_options)


NameError: name 'path' is not defined